In [3]:
import pandas as pd
import numpy as np
import requests
import json

In [4]:
# 所有高德API key，仅剩2000那个可用了
key_list = [
    'c4883baa101420590108e9ce0a2cfedd', # 0 吴泽宇 30000 封禁
    '3e2acbe01c50a83bc5a44b47bde7a858',
    'b51c10debdc93f34fb7b394bb866d8ea',
    '1c8507536f43a843bd25b9007eb638b5',
    '2355ca58db9271a092fd1fbc9be0e818',
    '5d4b115272a3ab7e871d1a719012b2a2', # 5 unknown 30000 封禁
    '7e3ca10787de5a314b6d85c6030b31e3',
    '3b90335174433a8a08014077a16ab202',
    '4dd40b220165ad9cd73d0c829b377d8a',
    '630f9c0cfe7e7dbad380fd447d268fcd',
    '9580b7a69f1f05dcd74d32e349c5bc0b', # 10 unknown 30000 封禁
    '8f5ae1c47298ba3954f05e6b48effdb1',
    '83853b0648f942f8fc3833d0de260976',
    '999450bcf9bf1f545ca3e3a9bc105ca1',
    'cb90b8b2e5083286c4fd5336e0554ce5',
    '34a92ac53696fc9dfc47cbdf70922449', # 15 ckr 2000 正常
    '714313e75688637245b9df2e87b3c617', # 16 19901854326 2000 正常
]

In [10]:
# 测试 key 是否被封
pa = {
        'key': key_list[16], #从控制台申请
        'city': '上海',
        'keywords': '宠物店',
        'page': 1,
        'offset': 25,
        'extensions': 'base'
}
r = requests.get('https://restapi.amap.com/v3/place/text?', params=pa)
decodejson = json.loads(r.text)
decodejson

{'suggestion': {'keywords': [], 'cities': []},
 'count': '891',
 'infocode': '10000',
 'pois': [{'parent': [],
   'address': '双桥路970号(靠近永辉生活)',
   'distance': [],
   'biz_ext': [],
   'pname': '上海市',
   'importance': [],
   'biz_type': [],
   'cityname': '上海市',
   'type': '购物服务;专卖店;宠物用品店',
   'photos': [],
   'typecode': '061211',
   'shopinfo': '0',
   'poiweight': [],
   'childtype': [],
   'adname': '浦东新区',
   'name': '道格宠物生活',
   'location': '121.609300,31.284322',
   'tel': '13524698280',
   'shopid': [],
   'id': 'B0FFJ1Y19B'},
  {'parent': [],
   'address': '临平路44号(瑞虹坊对面)',
   'distance': [],
   'biz_ext': [],
   'pname': '上海市',
   'importance': [],
   'biz_type': [],
   'cityname': '上海市',
   'type': '购物服务;购物相关场所;购物相关场所',
   'photos': [],
   'typecode': '060000',
   'shopinfo': '0',
   'poiweight': [],
   'childtype': [],
   'adname': '虹口区',
   'name': 'ANGELA私宠美容寄养精品店',
   'location': '121.498210,31.260701',
   'tel': '021-55830370',
   'shopid': [],
   'id': 'B0FFIIPN5J'},
  {

In [5]:
# 爬道路的途经点（没用）
pa = {
    'key': key_list[15], #从控制台申请
    'city': '021',
    'keywords': '莲花南路',
}
r = requests.get('http://restapi.amap.com/v3/road/roadname?', params=pa)
decodejson = json.loads(r.text)
try:
    print(r.text)
except Exception as e:
    print(e)

{"info":"INSUFFICIENT_PRIVILEGES","infocode":"10012","status":"0","sec_code_debug":"d41d8cd98f00b204e9800998ecf8427e","key":"34a92ac53696fc9dfc47cbdf70922449","sec_code":"d41d8cd98f00b204e9800998ecf8427e"}


In [12]:
# 根据关键字查询，例如：道路
poi_list = []
city_list = [
    '310101',
    '310104',
    '310105',
    '310106',
    '310107',
    '310109',
    '310110',
    '310112',
    '310113',
    '310114',
    '310115',
    '310116',
    '310117',
    '310118',
    '310120',
    '310151',
]

def get_poi_list(city, page):
    global poi_list
    pa = {
        'key': key_list[15], #从控制台申请
        'city': city,
        'keywords': '宠物店',
        'page': page,
        'offset': 25,
        'extensions': 'base'
    }
    r = requests.get('https://restapi.amap.com/v3/place/text?', params=pa)
    decodejson = json.loads(r.text)
    try:
        pois = decodejson['pois']
        if(len(pois)):
            poi_list += pois
            get_poi_list(city, page+1) # 递归
        else: 
            return
    except Exception as e:
        print(e)

for city in city_list:
    get_poi_list(city, 1)
df = pd.DataFrame(poi_list)[['id','name','type','typecode','address','location','pname','cityname','adname']]
df.set_index('id').to_excel('./数据/上海宠物店.xlsx')

In [ ]:
# 云南高铁站、机场
poi_list = []

def get_poi_list(city, types, keywords, page, key):
    global poi_list
    pa = {
        'key': key, #从控制台申请
        'city': city,
        'types': types,
        'keywords': keywords,
        'page': page,
        'offset': 25,
        'extensions': 'base'
    }
    r = requests.get('https://restapi.amap.com/v3/place/text?', params=pa)
    decodejson = json.loads(r.text)
    try:
        pois = decodejson['pois']
        if(len(pois)):
            poi_list += pois
            get_poi_list(city, types, keywords, page+1, key) # 递归
        else: 
            print(city, types, len(poi_list)) # 这一类查完了，没有下一页数据
            return
    except Exception as e:
        print(decodejson)
        print(e)

get_poi_list('云南省', '交通设施服务', '高铁站|机场', 1, '34a92ac53696fc9dfc47cbdf70922449')

df = pd.DataFrame(poi_list)[['id','name','type','typecode','address','location','pname','cityname','adname']]
df.set_index('id').to_excel('./数据/云南省高铁站机场.xlsx')

In [26]:
# 多城市 多类型查询
city_list = [
    #'保定市',
    #'秦皇岛市',
    #'石家庄市',
    #'邯郸市',
    #'唐山市',
    #'承德市',
    #'张家口市',
    '邢台市',
    '廊坊市',
    '沧州市',
]
small_types = [
    '餐饮服务','风景名胜','公共设施','公司企业','购物服务',
    '交通设施服务',
    '金融保险服务',
    '科教文化服务',
    '商务住宅','生活服务','体育休闲服务',
    '医疗保健服务','政府机构及社会团体'
]
poi_list = []

def get_poi_list(city, types, page, key):
    global poi_list
    pa = {
        'key': key, #从控制台申请
        'city': city,
        'types': types,
        #'keywords': '西丽街道',
        'page': page,
        'offset': 25,
        'extensions': 'base'
    }
    r = requests.get('https://restapi.amap.com/v3/place/text?', params=pa)
    decodejson = json.loads(r.text)
    try:
        pois = decodejson['pois']
        if(len(pois)):
            poi_list += pois
            get_poi_list(city, types, page+1, key) # 递归
        else: 
            print(city, types, len(poi_list)) # 这一类查完了，没有下一页数据
            return
    except Exception as e:
        print(e)

for idx, city in enumerate(city_list):
    for types in small_types:
        get_poi_list(city, types, 1, key_list[16])
        
df = pd.DataFrame(poi_list)[['id','name','type','typecode','address','location','pname','cityname','adname']]
df['lon'] = df['location'].apply(lambda x: x.split(',')[0])
df['lat'] = df['location'].apply(lambda x: x.split(',')[1])

df.set_index('id').to_excel('./数据/河北poi6.xlsx')

邢台市 餐饮服务 900
邢台市 风景名胜 1799
邢台市 公共设施 2699
邢台市 公司企业 3599
邢台市 购物服务 4499
邢台市 交通设施服务 5399
邢台市 金融保险服务 6299
邢台市 科教文化服务 7199
邢台市 商务住宅 8099
邢台市 生活服务 8999
邢台市 体育休闲服务 9899
邢台市 医疗保健服务 10799
邢台市 政府机构及社会团体 11699
廊坊市 餐饮服务 12598
廊坊市 风景名胜 13127
廊坊市 公共设施 14027
廊坊市 公司企业 14926
廊坊市 购物服务 15826
廊坊市 交通设施服务 16726
廊坊市 金融保险服务 17626
廊坊市 科教文化服务 18526
廊坊市 商务住宅 19426
廊坊市 生活服务 20326
廊坊市 体育休闲服务 21226
廊坊市 医疗保健服务 22126
廊坊市 政府机构及社会团体 23026
沧州市 餐饮服务 23926
沧州市 风景名胜 24552
沧州市 公共设施 25452
沧州市 公司企业 26352
沧州市 购物服务 27252
沧州市 交通设施服务 28152
沧州市 金融保险服务 29052
沧州市 科教文化服务 29952
沧州市 商务住宅 30852
沧州市 生活服务 31752
沧州市 体育休闲服务 32652
沧州市 医疗保健服务 33551
沧州市 政府机构及社会团体 34451


In [115]:
# 经纬度转平面坐标
def millerToXY (lon, lat):
    """
    :param lon: 经度
    :param lat: 维度
    :return:
    """
    L = 6381372*math.pi*2    #地球周长
    W = L                    #平面展开，将周长视为X轴
    H = L/2                  #Y轴约等于周长一般
    mill = 2.3               #米勒投影中的一个常数，范围大约在正负2.3之间  
    x = lon*math.pi/180      #将经度从度数转换为弧度
    y = lat*math.pi/180      #将纬度从度数转换为弧度 
    y = 1.25*math.log(math.tan(0.25*math.pi+0.4*y))  #这里是米勒投影的转换 
    
    # 这里将弧度转为实际距离 ，转换结果的单位是公里
    x = (W/2)+(W/(2*math.pi))*x
    y = (H/2)-(H/(2*mill))*y
    return int(round(x)), int(round(y))
df = pd.read_excel('./数据/深圳南山区西丽街道poi.xlsx')
df['平面坐标'] = df['location'].apply(lambda x: millerToXY(float(x.split(',')[0]), float(x.split(',')[1])))
df.set_index('id').to_excel('./数据/深圳南山区西丽街道poi.xlsx')
#millerToXY(116.420485, 39.897535)

In [71]:
# GCJ02/谷歌、高德 转换为 WGS84 gcj02towgs84
import math
def gcj02towgs84(localStr):
    lng = float(localStr.split(',')[0])
    lat = float(localStr.split(',')[1])
    PI = 3.1415926535897932384626
    ee = 0.00669342162296594323
    a = 6378245.0
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * PI
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * PI)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * PI)
    mglat = lat + dlat
    mglng = lng + dlng
    return str(lng * 2 - mglng) + ',' + str(lat * 2 - mglat)
def transformlat(lng, lat):
    PI = 3.1415926535897932384626
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * \
          lat + 0.1 * lng * lat + 0.2 * math.sqrt(abs(lng))
    ret += (20.0 * math.sin(6.0 * lng * PI) + 20.0 *
            math.sin(2.0 * lng * PI)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * PI) + 40.0 *
            math.sin(lat / 3.0 * PI)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * PI) + 320 *
            math.sin(lat * PI / 30.0)) * 2.0 / 3.0
    return ret
def transformlng(lng, lat):
    PI = 3.1415926535897932384626
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(abs(lng))
    ret += (20.0 * math.sin(6.0 * lng * PI) + 20.0 *
            math.sin(2.0 * lng * PI)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * PI) + 40.0 *
            math.sin(lng / 3.0 * PI)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * PI) + 300.0 *
            math.sin(lng / 30.0 * PI)) * 2.0 / 3.0
    return ret
df2['location_wgs84'] = df['location'].apply(gcj02towgs84)
df2

,id,name,type,typecode,address,location,pname,cityname,adname,location_wgs84
0,B0FFJDGJLN,Q泊士(国瑞购物中心C座),汽车服务;汽车服务相关;汽车服务相关,010000,国瑞西路与西花市大街交叉口西北50米,"116.420485,39.897535",北京市,北京市,东城区,"116.41425303550614,39.89613860380796"
1,B0FFGOLZ7M,水尔洁汽车服务分公司,汽车服务;汽车服务相关;汽车服务相关,010000,仓夹道7号楼地下停车场(洗车厂)(华普西门对面地下洗车厂),"116.431516,39.935592",北京市,北京市,东城区,"116.425294651109,39.93420554041404"
2,B0FFFW4MPF,东直门公共自行车租车服务站点,汽车服务;汽车服务相关;汽车服务相关,010000,东直门站西北角,"116.433053,39.941815",北京市,北京市,东城区,"116.42683354523544,39.94043032233963"
3,B0FFFW4MLP,卡迪卡汽车清洁美容专家,汽车服务;汽车服务相关;汽车服务相关|生活服务;生活服务场所;生活服务场所,010000|070000,建国门内大街9号北京国际饭店B1楼,"116.428330,39.909940",北京市,北京市,东城区,"116.42210675753992,39.9085507195"
4,B0FFIKK4OS,张宽养车(广渠门店),汽车服务;汽车服务相关;汽车服务相关,010000,广渠门外大街广渠家园11号楼B2,"116.449227,39.892679",北京市,北京市,东城区,"116.44304942592642,39.89132364719203"
...,...,...,...,...,...,...,...,...,...,...
230218,B0FFHCAY0I,华贸广场(北出入口),通行设施;虚拟门;虚拟门,991500,西大望路甲6号华贸广场,"116.480241,39.911015",北京市,北京市,朝阳区,"116.48522646913455,39.912357327578285"
230219,B0FFHCOXA6,金融街广场(东入口),通行设施;虚拟门;虚拟门,991500,金融大街35号金融街广场,"116.357550,39.910208",北京市,北京市,西城区,"116.44557026749783,39.90581813401757"
230220,B0FFHCD0G8,金融街广场(东出入口),通行设施;虚拟门;虚拟门,991500,金融大街35号金融街广场,"116.357599,39.910241",北京市,北京市,西城区,"116.46279655862979,39.88258273619985"
230221,B0FFHCOXA5,金融街广场(东北2入口),通行设施;虚拟门;虚拟门,991500,金融大街35号金融街广场,"116.357899,39.911097",北京市,北京市,西城区,"116.46078988558378,39.866899368963644"


In [77]:
df2['大类'] = df['type'].apply(lambda x: x.split(';')[0])
df2.sort_values(by='大类', inplace=True)
df2.head(2)

,id,name,type,typecode,address,location,pname,cityname,adname,location_wgs84,大类
449838,B0FFGYL9Y5,印象莫奈:时光映迹艺术展北京站,事件活动;公众活动;展会展览,220102,复兴路甲9号中华世纪坛内,"116.321222,39.911222",北京市,北京市,海淀区,"116.31510317387576,39.90992347230872",事件活动
520817,B0FFHQ5GOE,2019北京·通州运河文化庙会,事件活动;事件活动;事件活动,220000,芙蓉西路西150米,"116.680133,39.913767",北京市,北京市,通州区,"116.67418126835948,39.9126198383843",事件活动


In [80]:
# 分表
big_types_list = df2['大类'].unique().tolist()
for types in big_types_list:
    df2[df2['大类']==types].set_index('id').to_excel('./数据/北京poi分类/' + types + '.xlsx')